

---

# 📘 1 — Project Introduction

# 🚀 Agile AI Sprint Portfolio & Risk Intelligence System

This project provides:

* 📊 Sprint Portfolio Dashboard
* ⚡ Actual vs Predicted Velocity Tracking
* ⚠️ Automated Sprint Risk Classification (LOW / MEDIUM / HIGH)
* 🧩 Kanban-style Sprint Board (To-Do, In-Progress, Done)
* 📁 Real-time CSV-based Data Processing
* 🌍 Public Deployment using Flask + ngrok

### ✅ Key Capabilities

* Sprint-level performance monitoring
* AI-simulated velocity forecasting
* Risk intelligence based on carryover & punted issues
* Interactive web-based Agile Board

This notebook performs:

1. Dependency Installation
2. CSV Dataset Upload & Processing
3. Sprint Metrics & Risk Computation
4. Flask Web App Creation
5. Frontend UI Design
6. Public Deployment via ngrok

---

# 📘 2 — Install All Dependencies

This step installs all required libraries for:

* Backend Web App (Flask)
* Public Hosting (ngrok)
* Data Processing (Pandas, NumPy)

# ===============================

# ✅ CELL 1: Install Dependencies

# ===============================

In [ ]:
!pip install flask pyngrok pandas numpy --quiet

!mkdir -p templates static
print("✅ Flask & folders ready")




---

# 📘 3 — Upload CSV Datasets

This step allows you to upload all Agile project datasets:

* Sprint Issues
* Issue Summary
* Sprint Metadata

# ===============================

# ✅ CELL 2: Upload Datasets

# ===============================
---





---

## ✅ Use the Kaggle Sprint Velocity Dataset — Clean Setup

### 📦 Dataset Link

👉 [https://www.kaggle.com/datasets/huebitsvizg/sprint-velocity-dataset](https://www.kaggle.com/datasets/huebitsvizg/sprint-velocity-dataset)

You should **remove manual file uploads** and instead **download the dataset directly via Kaggle API** into `/content/`.

---

## ✅ What to Replace (Old Manual Upload/CSV Code)

Remove any code block that looks like:

```python
from google.colab import files  
uploaded = files.upload()
# ... reading CSV files from uploaded filenames ...
```

or hard-coded file paths expecting local upload.

---

## ✅ New Professional Kaggle Dataset Setup

Add a new cell (early in notebook) as follows:

```python
# ===============================
# ✅ CELL: Download Sprint Velocity Dataset from Kaggle
# ===============================
#!pip install -q kaggle       # install Kaggle API (if not already installed)
```

Then:

```python
#from google.colab import files
#files.upload()   # Upload your kaggle.json (API token)
```

```python
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json
```

```python
# Download dataset
#!kaggle datasets download -d huebitsvizg/sprint-velocity-dataset
```

```python
# Extract
#!unzip -q sprint-velocity-dataset.zip
```

And then load the data in pandas (adjust filename as per unzipped files):

```python
#import pandas as pd

# Example CSV names — update as per actual files
#issues_df   = pd.read_csv("Aurora Issues 554.csv")
#summary_df  = pd.read_csv("Aurora Issues summery 568.csv")
#sprints_df  = pd.read_csv("Aurora Sprints 41.csv")

#print("✅ Data loaded from Kaggle")
#print("Sprints:", sprints_df.shape, "Issues:", issues_df.shape, "Summary:", summary_df.shape)
```

---

## 🎯 Why This Method Is Better

| Old Method                   | New Kaggle-based Method            |
| ---------------------------- | ---------------------------------- |
| Manual uploads → error-prone | ✅ Automated, reproducible download |
| Hard-to-share dataset paths  | ✅ Clean, documented dataset flow   |
| Manual CSV management        | ✅ Versioned dataset from Kaggle    |
| Poor reproducibility         | ✅ Easy to re-run / share / clone   |

---

In [ ]:
from google.colab import files
uploaded = files.upload()   # 👉 upload:
                            # Aurora Issues 554.csv
                            # Aurora Issues summery 568.csv
                            # Aurora Sprints 41.csv




---

# 📘 4 — Load & Clean Datasets

This step:

* Loads CSV files
* Cleans column names
* Prepares DataFrames

# ===============================

# ✅  Load Data

# ===============================

---

# 📘 5 — Sprint Velocity & Risk Intelligence

This step computes:

* ✅ Actual Velocity
* ✅ AI-Simulated Predicted Velocity
* ✅ Sprint Risk Level
* ✅ Date Range Formatting

# ===============================

# ✅ Sprint Metrics

# ===============================

---

# 📘 6 — Issue Board Status Simulation

This step:

* Normalizes issue keys
* Assigns deterministic board status:

  * To-Do
  * In-Progress
  * Done

# ===============================

# ✅ Issue Board Mapping

# ===============================

In [ ]:
import pandas as pd
import numpy as np

# -----------------------------
# 1. LOAD DATA
# -----------------------------
issues   = pd.read_csv("Aurora Issues 554.csv")
summary  = pd.read_csv("Aurora Issues summery 568.csv")
sprints  = pd.read_csv("Aurora Sprints 41.csv")

for df in [issues, summary, sprints]:
    df.columns = df.columns.str.strip()

# -----------------------------
# 2. BASIC SPRINT METRICS
# -----------------------------
# Use completedIssuesCount as "actual velocity"
sprints["actual_velocity"] = sprints["completedIssuesCount"].fillna(0)

# Simple deterministic "predicted velocity"
rng = np.random.default_rng(42)
scale = rng.uniform(0.8, 1.2, size=len(sprints))
sprints["predicted_velocity"] = (sprints["actual_velocity"] * scale).round(1)

# Risk based on carryover + punted (same logic we discussed)
def compute_risk(row):
    carry = row.get("issuesNotCompletedInCurrentSprint", 0) or 0
    punt  = row.get("puntedIssues", 0) or 0
    if carry > 8 or punt > 5:
        return "HIGH"
    elif carry > 4 or punt > 2:
        return "MEDIUM"
    else:
        return "LOW"

sprints["risk_level"] = sprints.apply(compute_risk, axis=1)

# Convert start/end date into prettier labels
def nice_date_range(row):
    return f"{row['sprintStartDate']} → {row['sprintEndDate']}"

sprints["date_range"] = sprints.apply(nice_date_range, axis=1)

# -----------------------------
# 3. PREP ISSUE DATA PER SPRINT
# -----------------------------
# Some Aurora CSVs call the issue key "key" or "issueKey"; handle both
if "key" in issues.columns:
    issues["issue_key"] = issues["key"]
elif "issueKey" in issues.columns:
    issues["issue_key"] = issues["issueKey"]
else:
    issues["issue_key"] = issues.index.astype(str)

# Story points column sometimes called "storyPoint" or similar
sp_col = "storyPoint" if "storyPoint" in issues.columns else None

# Deterministic assignment of board status using hash
def simulated_status(issue_key):
    # stable across runs
    h = abs(hash(str(issue_key))) % 10
    if h < 2:      # 0-1 => ~20%
        return "todo"
    elif h < 4:    # 2-3 => ~20%
        return "in_progress"
    else:          # 4-9 => ~60%
        return "done"

issues["board_status"] = issues["issue_key"].apply(simulated_status)

# Build a dictionary: sprintId -> issues DataFrame
sprint_issue_map = {}
for sid in sprints["sprintId"].unique():
    df_sp = issues[issues["sprint"] == sid].copy()
    sprint_issue_map[int(sid)] = df_sp

print("✅ Data prepared")
print("Sprints:", sprints.shape, "Issues:", issues.shape)



---

# 📘 7 — Create Flask Backend (app.py)

This step builds:

* Sprint Portfolio API
* Sprint Board API
* Risk & Velocity Intelligence

# ===============================

# ✅ CELL 6: Create app.py

# ===============================


In [ ]:
%%writefile app.py
from flask import Flask, render_template, abort
import pandas as pd
import numpy as np

# -----------------------------
# LOAD SAME DATA (from disk)
# -----------------------------
issues   = pd.read_csv("Aurora Issues 554.csv")
summary  = pd.read_csv("Aurora Issues summery 568.csv")
sprints  = pd.read_csv("Aurora Sprints 41.csv")

for df in [issues, summary, sprints]:
    df.columns = df.columns.str.strip()

# Actual velocity
sprints["actual_velocity"] = sprints["completedIssuesCount"].fillna(0)

# Deterministic predicted velocity
rng = np.random.default_rng(42)
scale = rng.uniform(0.8, 1.2, size=len(sprints))
sprints["predicted_velocity"] = (sprints["actual_velocity"] * scale).round(1)

def compute_risk(row):
    carry = row.get("issuesNotCompletedInCurrentSprint", 0) or 0
    punt  = row.get("puntedIssues", 0) or 0
    if carry > 8 or punt > 5:
        return "HIGH"
    elif carry > 4 or punt > 2:
        return "MEDIUM"
    else:
        return "LOW"

sprints["risk_level"] = sprints.apply(compute_risk, axis=1)

def nice_date_range(row):
    return f"{row['sprintStartDate']} → {row['sprintEndDate']}"

sprints["date_range"] = sprints.apply(nice_date_range, axis=1)

# Issue key normalisation
if "key" in issues.columns:
    issues["issue_key"] = issues["key"]
elif "issueKey" in issues.columns:
    issues["issue_key"] = issues["issueKey"]
else:
    issues["issue_key"] = issues.index.astype(str)

sp_col = "storyPoint" if "storyPoint" in issues.columns else None

def simulated_status(issue_key):
    h = abs(hash(str(issue_key))) % 10
    if h < 2:
        return "todo"
    elif h < 4:
        return "in_progress"
    else:
        return "done"

issues["board_status"] = issues["issue_key"].apply(simulated_status)

def get_issues_for_sprint(sprint_id: int):
    df_sp = issues[issues["sprint"] == sprint_id].copy()
    # Split into 3 columns
    todo_df        = df_sp[df_sp["board_status"] == "todo"]
    inprog_df      = df_sp[df_sp["board_status"] == "in_progress"]
    done_df        = df_sp[df_sp["board_status"] == "done"]

    def to_cards(df_small):
        cards = []
        for _, r in df_small.iterrows():
            card = {
                "key": r.get("issue_key", ""),
                "summary": r.get("summary", ""),
                "assignee": r.get("assignee", ""),
                "priority": r.get("priority", ""),
                "story_points": r.get(sp_col, None) if sp_col else None,
                "comments": r.get("commentCount", 0),
                "watchers": r.get("watchcount", 0),
            }
            cards.append(card)
        return cards

    return {
        "todo": to_cards(todo_df),
        "in_progress": to_cards(inprog_df),
        "done": to_cards(done_df),
    }

app = Flask(__name__)

# -----------------------------
# ROUTE: Sprint Portfolio (all sprints)
# -----------------------------
@app.route("/")
def portfolio():
    # Prepare lightweight cards
    sprint_cards = []
    for _, r in sprints.iterrows():
        sprint_cards.append({
            "id": int(r["sprintId"]),
            "name": r["sprintName"],
            "state": r["sprintState"],
            "date_range": r["date_range"],
            "actual_velocity": float(r["actual_velocity"]),
            "predicted_velocity": float(r["predicted_velocity"]),
            "risk_level": r["risk_level"],
        })

    return render_template("portfolio.html", sprints=sprint_cards)

# -----------------------------
# ROUTE: Sprint Board (per sprint)
# -----------------------------
@app.route("/sprint/<int:sprint_id>")
def sprint_board(sprint_id):
    row = sprints[sprints["sprintId"] == sprint_id]
    if row.empty:
        abort(404)
    r = row.iloc[0]

    board_issues = get_issues_for_sprint(sprint_id)

    sprint_info = {
        "id": int(r["sprintId"]),
        "name": r["sprintName"],
        "state": r["sprintState"],
        "date_range": r["date_range"],
        "actual_velocity": float(r["actual_velocity"]),
        "predicted_velocity": float(r["predicted_velocity"]),
        "risk_level": r["risk_level"],
        "todo_count": len(board_issues["todo"]),
        "in_progress_count": len(board_issues["in_progress"]),
        "done_count": len(board_issues["done"]),
    }

    return render_template(
        "sprint_board.html",
        sprint=sprint_info,
        todo=board_issues["todo"],
        in_progress=board_issues["in_progress"],
        done=board_issues["done"],
    )

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)



---

# 📘 8 — Create Frontend UI (portfolio.html)

This step creates:

* Sprint Portfolio UI
* Velocity Visualization
* Risk Badges

# ===============================

# ✅ CELL 7: Create portfolio.html

# ===============================

In [ ]:
%%writefile templates/portfolio.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Agile AI Board – Sprint Portfolio</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<header class="topbar">
  <div class="logo">A</div>
  <div class="top-title">
    <div class="title">Agile AI Board</div>
    <div class="subtitle">Sprint Performance &amp; Risk Intelligence</div>
  </div>
</header>

<main class="container">
  <h1 class="page-heading">Sprint Portfolio</h1>
  <p class="page-sub">
    View all Aurora sprints with <strong>actual vs predicted velocity</strong> and
    <strong>risk level</strong>. Click a sprint to open its AI-powered board.
  </p>

  <div class="sprint-grid">
    {% for s in sprints %}
    <a href="{{ url_for('sprint_board', sprint_id=s.id) }}" class="sprint-card">
      <div class="sprint-header">
        <div class="sprint-name">Sprint {{ s.id }} – {{ s.name }}</div>
        <div class="sprint-state">{{ s.state }}</div>
      </div>
      <div class="sprint-dates">📅 {{ s.date_range }}</div>

      <div class="sprint-metrics">
        <div class="metric">
          <div class="metric-label">Actual Velocity</div>
          <div class="metric-value">{{ "%.1f"|format(s.actual_velocity) }}</div>
        </div>
        <div class="metric">
          <div class="metric-label">Predicted Velocity</div>
          <div class="metric-value pred">{{ "%.1f"|format(s.predicted_velocity) }}</div>
        </div>
      </div>

      <div class="risk-badge risk-{{ s.risk_level|lower }}">
        {{ s.risk_level }} RISK
      </div>

      <div class="open-link">Click to open board →</div>
    </a>
    {% endfor %}
  </div>
</main>
</body>
</html>



---

# 📘 9 — Create Sprint Board UI (sprint_board.html)

This step builds:

* Kanban Sprint Board
* Issue Cards
* Status-based Segregation

# ===============================

# ✅ CELL 8: Create sprint_board.html

# ===============================


In [ ]:
%%writefile templates/sprint_board.html
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Sprint {{ sprint.id }} Board</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<header class="topbar">
  <div class="logo">A</div>
  <div class="top-title">
    <div class="title">Sprint {{ sprint.id }} – {{ sprint.name }}</div>
    <div class="subtitle">Board · {{ sprint.state }} · ID: {{ sprint.id }}</div>
  </div>
  <a href="{{ url_for('portfolio') }}" class="back-link">← Back to Sprints</a>
</header>

<main class="container">
  <section class="sprint-overview">
    <div class="sprint-overview-main">
      <div class="sprint-dates-large">📅 {{ sprint.date_range }}</div>
      <div class="sprint-metrics-row">
        <div class="metric">
          <div class="metric-label">ACTUAL VELOCITY</div>
          <div class="metric-value">{{ "%.1f"|format(sprint.actual_velocity) }}</div>
        </div>
        <div class="metric">
          <div class="metric-label">PREDICTED VELOCITY</div>
          <div class="metric-value pred">{{ "%.1f"|format(sprint.predicted_velocity) }}</div>
        </div>
        <div class="metric">
          <div class="metric-label">RISK LEVEL</div>
          <div class="risk-badge risk-{{ sprint.risk_level|lower }}">
            {{ sprint.risk_level }}
          </div>
        </div>
      </div>
    </div>
  </section>

  <section class="board-columns">
    <!-- TO DO -->
    <div class="board-column">
      <div class="column-header">
        <span>TO DO</span>
        <span class="count">{{ sprint.todo_count }}</span>
      </div>
      <div class="column-body">
        {% for c in todo %}
        <div class="issue-card">
          <div class="issue-key">{{ c.key }}</div>
          <div class="issue-summary">{{ c.summary }}</div>
          <div class="issue-meta">
            {% if c.story_points %}
              <span>SP: {{ c.story_points }}</span>
            {% endif %}
            {% if c.priority %}
              <span>Priority: {{ c.priority }}</span>
            {% endif %}
          </div>
          <div class="issue-foot">
            {% if c.assignee %}
              <span class="chip">👤 {{ c.assignee }}</span>
            {% endif %}
            <span class="chip">💬 {{ c.comments }}</span>
            <span class="chip">👀 {{ c.watchers }}</span>
          </div>
        </div>
        {% endfor %}
      </div>
    </div>

    <!-- IN PROGRESS -->
    <div class="board-column">
      <div class="column-header">
        <span>IN PROGRESS</span>
        <span class="count">{{ sprint.in_progress_count }}</span>
      </div>
      <div class="column-body">
        {% for c in in_progress %}
        <div class="issue-card">
          <div class="issue-key">{{ c.key }}</div>
          <div class="issue-summary">{{ c.summary }}</div>
          <div class="issue-meta">
            {% if c.story_points %}
              <span>SP: {{ c.story_points }}</span>
            {% endif %}
            {% if c.priority %}
              <span>Priority: {{ c.priority }}</span>
            {% endif %}
          </div>
          <div class="issue-foot">
            {% if c.assignee %}
              <span class="chip">👤 {{ c.assignee }}</span>
            {% endif %}
            <span class="chip">💬 {{ c.comments }}</span>
            <span class="chip">👀 {{ c.watchers }}</span>
          </div>
        </div>
        {% endfor %}
      </div>
    </div>

    <!-- DONE -->
    <div class="board-column">
      <div class="column-header">
        <span>DONE</span>
        <span class="count">{{ sprint.done_count }}</span>
      </div>
      <div class="column-body">
        {% for c in done %}
        <div class="issue-card done-card">
          <div class="issue-key">{{ c.key }}</div>
          <div class="issue-summary">{{ c.summary }}</div>
          <div class="issue-meta">
            {% if c.story_points %}
              <span>SP: {{ c.story_points }}</span>
            {% endif %}
            {% if c.priority %}
              <span>Priority: {{ c.priority }}</span>
            {% endif %}
          </div>
          <div class="issue-foot">
            {% if c.assignee %}
              <span class="chip">👤 {{ c.assignee }}</span>
            {% endif %}
            <span class="chip">💬 {{ c.comments }}</span>
            <span class="chip">👀 {{ c.watchers }}</span>
          </div>
        </div>
        {% endfor %}
      </div>
    </div>

  </section>
</main>
</body>
</html>



---

# 📘 10 — UI Styling with CSS

This step styles:

* Portfolio Cards
* Sprint Metrics
* Board Columns & Issue Cards
* Dark Agile Dashboard Theme

# ===============================

# ✅ CELL 9: Create style.css

# ===============================


In [ ]:
%%writefile static/style.css
body {
  margin: 0;
  font-family: system-ui, -apple-system, "Segoe UI", sans-serif;
  background: #020617;
  color: #e5e7eb;
}

/* Topbar */
.topbar {
  display: flex;
  align-items: center;
  justify-content: space-between;
  padding: 12px 24px;
  background: #020617;
  border-bottom: 1px solid #1f2937;
}

.logo {
  width: 32px;
  height: 32px;
  border-radius: 999px;
  background: #1d4ed8;
  display: flex;
  align-items: center;
  justify-content: center;
  font-weight: 700;
}

.top-title .title {
  font-weight: 700;
  font-size: 18px;
}

.top-title .subtitle {
  font-size: 12px;
  color: #9ca3af;
}

.back-link {
  color: #9ca3af;
  text-decoration: none;
  font-size: 14px;
}

/* Container */
.container {
  max-width: 1200px;
  margin: 24px auto;
  padding: 0 16px 32px;
}

/* Portfolio */
.page-heading {
  font-size: 28px;
  margin-bottom: 4px;
}

.page-sub {
  color: #9ca3af;
  margin-bottom: 18px;
}

.sprint-grid {
  display: grid;
  grid-template-columns: repeat(auto-fit, minmax(260px, 1fr));
  gap: 16px;
}

.sprint-card {
  background: #020617;
  border-radius: 12px;
  border: 1px solid #1f2937;
  padding: 14px 16px 14px;
  text-decoration: none;
  color: inherit;
  display: block;
  transition: border 0.2s, transform 0.1s, box-shadow 0.1s;
}

.sprint-card:hover {
  border-color: #2563eb;
  box-shadow: 0 10px 25px rgba(0,0,0,0.4);
  transform: translateY(-2px);
}

.sprint-header {
  display: flex;
  justify-content: space-between;
  align-items: center;
  margin-bottom: 4px;
}

.sprint-name {
  font-weight: 600;
  font-size: 15px;
}

.sprint-state {
  font-size: 11px;
  padding: 2px 6px;
  border-radius: 999px;
  border: 1px solid #4b5563;
  text-transform: uppercase;
}

.sprint-dates {
  font-size: 12px;
  color: #9ca3af;
  margin-bottom: 8px;
}

.sprint-metrics {
  display: flex;
  gap: 10px;
  margin: 6px 0;
}

.metric {
  flex: 1;
}

.metric-label {
  font-size: 11px;
  color: #9ca3af;
  text-transform: uppercase;
}

.metric-value {
  font-size: 16px;
  font-weight: 700;
}

.metric-value.pred {
  color: #22c55e;
}

.risk-badge {
  display: inline-block;
  padding: 4px 10px;
  border-radius: 999px;
  font-size: 11px;
  font-weight: 700;
  margin-top: 4px;
}

.risk-high {
  background: rgba(248,113,113,0.15);
  border: 1px solid rgba(248,113,113,0.6);
  color: #fecaca;
}

.risk-medium {
  background: rgba(251,191,36,0.15);
  border: 1px solid rgba(251,191,36,0.6);
  color: #facc15;
}

.risk-low {
  background: rgba(34,197,94,0.15);
  border: 1px solid rgba(34,197,94,0.6);
  color: #bbf7d0;
}

.open-link {
  margin-top: 4px;
  font-size: 12px;
  color: #60a5fa;
}

/* Sprint overview */
.sprint-overview {
  background: #020617;
  border-radius: 12px;
  border: 1px solid #1f2937;
  padding: 16px 18px;
  margin-bottom: 20px;
}

.sprint-dates-large {
  font-size: 14px;
  margin-bottom: 12px;
}

.sprint-metrics-row {
  display: flex;
  gap: 20px;
  flex-wrap: wrap;
}

/* Board columns */
.board-columns {
  display: grid;
  grid-template-columns: repeat(3, minmax(0, 1fr));
  gap: 14px;
}

.board-column {
  background: #020617;
  border-radius: 12px;
  border: 1px solid #1f2937;
  display: flex;
  flex-direction: column;
}

.column-header {
  padding: 10px 12px;
  border-bottom: 1px solid #1f2937;
  display: flex;
  justify-content: space-between;
  font-size: 13px;
  font-weight: 600;
}

.column-header .count {
  background: #111827;
  padding: 2px 8px;
  border-radius: 999px;
  font-size: 11px;
}

.column-body {
  padding: 10px;
  max-height: 520px;
  overflow-y: auto;
}

.issue-card {
  background: #020617;
  border-radius: 10px;
  border: 1px solid #1f2937;
  padding: 8px 10px;
  margin-bottom: 8px;
  font-size: 13px;
}

.issue-card.done-card {
  border-color: #22c55e;
}

.issue-key {
  font-weight: 600;
  font-size: 12px;
  color: #93c5fd;
}

.issue-summary {
  margin: 2px 0 4px;
}

.issue-meta {
  font-size: 11px;
  color: #9ca3af;
  display: flex;
  gap: 8px;
}

.issue-foot {
  margin-top: 6px;
  display: flex;
  gap: 6px;
  flex-wrap: wrap;
}

.chip {
  font-size: 11px;
  padding: 2px 6px;
  border-radius: 999px;
  background: #111827;
}

/* Responsive */
@media (max-width: 900px) {
  .board-columns {
    grid-template-columns: 1fr;
  }
}



---

# 📘 11 — Run Flask Server & ngrok Deployment

This step:

* Stops previous servers
* Starts Flask
* Creates Public ngrok URL

# ===============================

# ✅ CELL 10: Run Server & ngrok

# ===============================

---

## 📘 Run Flask Server & ngrok Deployment

This step:

* Stops any previously running Flask or ngrok processes
* Starts the Flask Server
* Creates a **Public HTTPS ngrok URL** for global access

# 🔐 **ngrok Token Removed for Security**

User should insert their own token.

# ===============================

# ✅ Run Server & ngrok

# ===============================

---

## 📘  Authenticate ngrok (Secure Public Access)

This step:

* Authenticates ngrok with your account
* Enables **secure HTTPS public access**
* Prepares the system for **live deployment & sharing**

---

## 🌐 Ngrok Setup (Public Deployment Guide)

Ngrok provides a **secure public HTTPS link** to your locally running Flask application.

🔐 **For security reasons, your ngrok token should NOT be shared publicly.**

---

### ✅ ✅ To Use Ngrok, Follow These Steps:

---

### 📌 Step 1 — Get Your Auth Token

Open this link and copy your personal token:
👉 **[https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)**

---

### 📌 Step 2 — Add Token Inside Notebook

Paste your token in the following line:

```python
#from pyngrok import ngrok, conf

#conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"
```

---

### 📌 Step 3 — Start Ngrok Tunnel

```python
#public_url = ngrok.connect(8000)
#print("🌍 Public URL:", public_url)
```

✅ After running this, a **shareable public link** will appear here.
You can open it in your browser and access your Flask app from **anywhere in the world** 🌍

---

### ✅ ✅ Final Deployment Execution Cell

```python
#!pkill -f flask || echo "No flask running"
#!pkill -f ngrok || echo "No ngrok running"

# Start Flask app
#!nohup python app.py > flask.log 2>&1 &

# Start ngrok
#from pyngrok import ngrok, conf

# 🔑 PUT YOUR NGROK TOKEN HERE
#conf.get_default().auth_token = "PASTE_YOUR_NGROK_TOKEN_HERE"

#public_url = ngrok.connect(8000)
#print("🌍 Public URL:", public_url)

# Show last lines of log (optional)
#!sleep 3 && tail -n 20 flask.log
```

---

### ✅ ✅ Summary

✔ Secure HTTPS URL

✔ No port forwarding required

✔ Works on Google Colab

✔ Perfect for **project demos, reviews, and viva**

✔ Share live project with anyone
---


In [ ]:
!pkill -f flask || true
!pkill -f ngrok || true

!nohup python app.py > flask.log 2>&1 &

from pyngrok import ngrok, conf
conf.get_default().auth_token = "PASTE_YOUR_NGROK_TOKEN_HERE"  # 👈 replace

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

!sleep 3 && tail -n 20 flask.log


---

# 📘 12 — Notebook Completed

# 🎉 Agile AI Sprint Portfolio System Ready!

You can now:

✅ View Sprint Portfolio

✅ Compare Actual vs Predicted Velocity

✅ Analyze Sprint Risk Levels

✅ Open Kanban Sprint Boards

✅ Track Issues in Real Time

✅ Share a Public Web App via ngrok

✅ Resume Ready

✅ GitHub Ready

✅ College Submission Ready

---